In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = = = = =

import pandas as pd
from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=http://u431.kdltps.com:15818')
option.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36')
option.page_load_strategy = 'eager'

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./menu.xlsx', header=0)
list_url = df_menu['Url'].to_list()

print('总数量：' + str(len(list_url)))
print()

work = Queue()
for url in list_url:
    work.put_nowait(url)

# = = = = = = = = = = = = = = = = = =

df = pd.DataFrame(columns=['Part_Number',
                           'Manufacturer',
                           'Vehicle',
                           'Note_1',
                           'Info',
                           'Note_2',
                           'Alternate_OE_Part_Numbers',
                           'Src',
                           'status'])

# = = = = = = = = = = = = = = = = = =

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        status = 'error'
        for _ in range(31):
            try:
                browser = Chrome(service=Service('../../../chromedriver'), options=option)
                browser.maximize_window()
                
                # = = = = = = = = = = = = = = = = = =

                browser.get(url)
                
                time.sleep(5)
                
                # = = = = = = = = = = = = = = = = = =
                
                if len(browser.find_elements(by=By.XPATH, value='//span[@class="navlabellink nvoffset nnormal"]')) != 0:
                    break
                    
                # = = = = = = = = = = = = = = = = = =

                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//tbody[@class="listing-inner altrow-a-1  "]')))
                list_part = browser.find_elements(by=By.XPATH, value='//tbody[@class="listing-inner altrow-a-1  "]') + browser.find_elements(by=By.XPATH, value='//tbody[@class="listing-inner altrow-a-0  "]')

                # = = = = = = = = = = = = = = = = = =
                
                list_part_number = [part.find_element(by=By.XPATH, value='./descendant::div[@class="listing-text-row-moreinfo-truck"]/span[@title="Buyer\'s Guide"]') for part in list_part]

                # = = = = = = = = = = = = = = = = = =
                
                part_index = [part_number.text for part_number in list_part_number].index(url.split(',')[-3])
                
                # = = = = = = = = = = = = = = = = = =
                
                manufacturer = list_part[part_index].find_element(by=By.XPATH, value='./descendant::div[@class="listing-text-row-moreinfo-truck"]/span[@class="listing-final-manufacturer "]').text

                # = = = = = = = = = = = = = = = = = =
                
                list_part_number[part_index].click()
                time.sleep(5)
                list_vehicle = browser.find_element(by=By.XPATH, value='//div[@id="buyersguidepopup-outer_c"]').find_elements(by=By.XPATH, value='./descendant::table[@class="nobmp"]')
                if len(list_vehicle) == 0:
                    vehicle = ''
                else:
                    vehicle = list_vehicle[0].text
    
                # = = = = = = = = = = = = = = = = = =
                
                note_1 = ';'.join(row.text for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]'))

                # = = = = = = = = = = = = = = = = = =

                info = ';'.join(row.get_attribute('href') for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]'))

                # = = = = = = = = = = = = = = = = = =
                
                note_2 = ';'.join(row.text for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[@class="listing-text-row"]'))

                # = = = = = = = = = = = = = = = = = =

                oe = ';'.join(row.get_attribute('textContent') for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::span[@title="Replaces these Alternate/ OE Part Numbers"]'))
                oe = ';'.join(oe.split(', '))
                
                # = = = = = = = = = = = = = = = = = =

                src = ';'.join(row.get_attribute('src') for row in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::img[@class=" listing-inline-image"]'))

                # = = = = = = = = = = = = = = = = = =
                
                status = 'ok'
                df_temp = pd.DataFrame([{'Part_Number': url.split(',')[-3],
                                         'Manufacturer': manufacturer,
                                         'Vehicle': vehicle,
                                         'Note_1': note_1,
                                         'Info': info,
                                         'Note_2': note_2,
                                         'Alternate_OE_Part_Numbers': oe,
                                         'Src': src,
                                         'status': 'ok'}])
                
                browser.quit()
                
                break
                
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': url.split(',')[-3],
                                     'status': 'error'}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url.split(',')[-3] +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(15):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df.to_excel('./part.xlsx', index=False)
print()
print('搞定')

总数量：429

2451018  <->  [ok] - 剩余数量：414
2451005  <->  [ok] - 剩余数量：413
2451006  <->  [ok] - 剩余数量：412
2451002  <->  [ok] - 剩余数量：411
2451017  <->  [ok] - 剩余数量：410
2451019  <->  [ok] - 剩余数量：409
2451015  <->  [ok] - 剩余数量：408
2451000  <->  [ok] - 剩余数量：407
2451013  <->  [ok] - 剩余数量：406
2451016  <->  [ok] - 剩余数量：405
2451014  <->  [ok] - 剩余数量：404
2451004  <->  [ok] - 剩余数量：403
2451012  <->  [ok] - 剩余数量：402
2451020  <->  [ok] - 剩余数量：401
2451024  <->  [ok] - 剩余数量：400
2451033  <->  [ok] - 剩余数量：399
2451036  <->  [ok] - 剩余数量：398
2451029  <->  [ok] - 剩余数量：397
2451026  <->  [ok] - 剩余数量：396
2451001  <->  [ok] - 剩余数量：395
2451003  <->  [ok] - 剩余数量：394
2451027  <->  [ok] - 剩余数量：393
2451021  <->  [ok] - 剩余数量：392
2451046  <->  [ok] - 剩余数量：391
2451047  <->  [ok] - 剩余数量：390
2451052  <->  [ok] - 剩余数量：389
2451051  <->  [ok] - 剩余数量：388
2451053  <->  [ok] - 剩余数量：387
2451045  <->  [ok] - 剩余数量：386
2451055  <->  [ok] - 剩余数量：385
2451043  <->  [ok] - 剩余数量：384
2451025  <->  [ok] - 剩余数量：383
2451058  <->  [ok] - 剩余数量：382
2

2451315  <->  [ok] - 剩余数量：141
2451407  <->  [ok] - 剩余数量：140
2451412  <->  [ok] - 剩余数量：139
2451398  <->  [ok] - 剩余数量：138
2451424  <->  [ok] - 剩余数量：137
2451425  <->  [ok] - 剩余数量：136
2451415  <->  [ok] - 剩余数量：135
2451433  <->  [ok] - 剩余数量：134
2451434  <->  [ok] - 剩余数量：133
2451430  <->  [ok] - 剩余数量：132
2451431  <->  [ok] - 剩余数量：131
2451467  <->  [ok] - 剩余数量：130
2451458  <->  [ok] - 剩余数量：129
2451417  <->  [ok] - 剩余数量：128
2451451  <->  [ok] - 剩余数量：127
2451418  <->  [ok] - 剩余数量：126
2451426  <->  [ok] - 剩余数量：125
2451474  <->  [ok] - 剩余数量：124
2451573  <->  [ok] - 剩余数量：123
2451481  <->  [ok] - 剩余数量：122
2451575  <->  [ok] - 剩余数量：121
2451583  <->  [ok] - 剩余数量：120
2451582  <->  [ok] - 剩余数量：119
2451587  <->  [ok] - 剩余数量：118
2451586  <->  [ok] - 剩余数量：117
2451613  <->  [ok] - 剩余数量：116
2451619  <->  [ok] - 剩余数量：115
2451529  <->  [ok] - 剩余数量：114
2451655  <->  [ok] - 剩余数量：113
2451574  <->  [ok] - 剩余数量：112
2452005  <->  [ok] - 剩余数量：111
2451599  <->  [ok] - 剩余数量：110
2452012  <->  [ok] - 剩余数量：109
2452013  <